<center><h1>Istanbul, Izmir House_Web Scriping</h1></center>


<div style="text-align: center;">
    <img src="https://miro.medium.com/v2/resize:fit:820/1*TC2JtooA4LoOauX34_fUWA.png" alt="web scraping">
</div>

Data on houses for sale in Istanbul and Izmir from the website emlaksitem.com has been collected using the web scraping method, with BeautifulSoup employed for parsing the HTML content.

In [2]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
from bs4 import BeautifulSoup
import requests
pd.set_option("display.max_columns",100)

In [1]:
def getAndParseURL(url):
    result = requests.get(url, headers={"User-Agent": "#User-Agent"})
    soup = BeautifulSoup(result.text, 'html.parser')
    return soup

## Istanbul city houses

In [ ]:
PAGE_NUM = []
def find_page_num():


    for i in range(1,18):
        PAGE_NUM.append("https://emlaksitem.com/satilik-daire/istanbul?page=" + str(i))
    return PAGE_NUM
find_page_num()

['https://emlaksitem.com/satilik-daire/istanbul?page=1',
 'https://emlaksitem.com/satilik-daire/istanbul?page=2',
 'https://emlaksitem.com/satilik-daire/istanbul?page=3',
 'https://emlaksitem.com/satilik-daire/istanbul?page=4',
 'https://emlaksitem.com/satilik-daire/istanbul?page=5',
 'https://emlaksitem.com/satilik-daire/istanbul?page=6',
 'https://emlaksitem.com/satilik-daire/istanbul?page=7',
 'https://emlaksitem.com/satilik-daire/istanbul?page=8',
 'https://emlaksitem.com/satilik-daire/istanbul?page=9',
 'https://emlaksitem.com/satilik-daire/istanbul?page=10',
 'https://emlaksitem.com/satilik-daire/istanbul?page=11',
 'https://emlaksitem.com/satilik-daire/istanbul?page=12',
 'https://emlaksitem.com/satilik-daire/istanbul?page=13',
 'https://emlaksitem.com/satilik-daire/istanbul?page=14',
 'https://emlaksitem.com/satilik-daire/istanbul?page=15',
 'https://emlaksitem.com/satilik-daire/istanbul?page=16',
 'https://emlaksitem.com/satilik-daire/istanbul?page=17']

In [ ]:
ALL_PRODUCT = []
def all_product():
    for i in PAGE_NUM[1:2:1]:
        html = getAndParseURL(i)
        for sonuc in html.findAll("a",{"class":"classified-title"}):
            ALL_PRODUCT.append("https://emlaksitem.com" + sonuc.get("href"))
    return ALL_PRODUCT
all_product()

['https://emlaksitem.com/ilan/bayatli-sitesinde-jeotermalli-satilik-573457',
 'https://emlaksitem.com/ilan/21-kemalreis-ucyolda-yatirimlik-95m2-yuksek-giris-esyali-satilik-daire-573505',
 'https://emlaksitem.com/ilan/elitten-yesilyurt-merkezde-satilik-cok-buyuk-daire-573504',
 'https://emlaksitem.com/ilan/izmir-torbali-bahcelievlerde-sifir-satilik-yatirimlik-daireler-21-90-m2-573467',
 'https://emlaksitem.com/ilan/dikili-ummetoglundan-11-55-m2-daire-250-m2-depo-satilik-yatirimlik-573424',
 'https://emlaksitem.com/ilan/urla-merkezde-yeni-binada-31-dublex-573487',
 'https://emlaksitem.com/ilan/31-satilik-daire-ertugrul-mahallesinde-573396',
 'https://emlaksitem.com/ilan/izmir-odemis-sdemirel-mah-satilik-onu-acik-dublex-daire-573406',
 'https://emlaksitem.com/ilan/odemis-bengisu-mah-satilik-cadde-uzeri-ara-kat-daire-573404',
 'https://emlaksitem.com/ilan/31-satilik-daire-torbali-da-573397',
 'https://emlaksitem.com/ilan/dikili-ummetogluemlaktan-satilik-giris-kat-21-daire-90-m2-573358',
 '

In [ ]:


def parse_data():
    rows = []  # We'll create a row for each product
    columns = set()  # Using a set to store column names

    for url in ALL_PRODUCT:
        result = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3.1 Safari/605.1.15"})
        soup = BeautifulSoup(result.text, 'html.parser')

        target_div = soup.find("div", {"id": "ilan-bilgileri"})
        row_data = {}  # We'll store row data for each URL

        if target_div:
            ul = target_div.find('ul')
            if ul:
                li_items = ul.find_all('li')

                for li in li_items:
                    strong_items = li.find_all("strong")
                    span_items = li.find('span')

                    if len(strong_items) == 2:
                        column_name = strong_items[0].text.strip()
                        value = strong_items[1].text.strip()
                    else:
                        column_name = strong_items[0].text.strip()
                        value = span_items.text.strip() if span_items else ''

                    row_data[column_name] = value
                    columns.add(column_name)  # Add column names to the set

        prices = soup.select('span.btn.btn-success')
        price_values = [price.text.strip() for price in prices]

        if price_values:
            row_data['Price'] = ', '.join(price_values)
            columns.add('Price')  # Add 'Price' column to the set

        # Add empty data for missing columns
        for column in columns:
            if column not in row_data:
                row_data[column] = ''

        rows.append(row_data)

    # Create DataFrame
    df = pd.DataFrame(rows, columns=sorted(columns))
    return df

df = parse_data()


In [ ]:
df.shape

(423, 21)

In [ ]:
df.head()

,Aidat (TL),Banyo Sayısı,Bina Yaşı,Bir Kattaki Daire,Bulunduğu Kat,Eşyalı,Gör. Arama İle Gez.,Isıtma,Kat Sayısı,Krediye Uygun,Kullanım Durumu,Kısıtlama,Metre Karesi (Net),Oda Sayısı,Price,Site İçerisinde,Takaslı,Yapı Kullanım İzin D.,İlan No,İlan Tarihi,İlan Türü
0,NaN,: 1,: 26-30,NaN,: 3,NaN,NaN,: Doğalgaz (Kombi),: 3,: Hayır,: Boş,: Yok,: 95m2,: 2+1,2.600.000 TL,NaN,: Hayır,NaN,: 29664799,: 31 Temmuz 24,: Satılık Daire
1,NaN,: 1,: 21-25,NaN,: Giriş Katı,NaN,NaN,: Kalorifer (Doğalgaz),: 6,: Hayır,: Mülk Sahibi,: Yok,: 75m2,: 2+1,1.700.000 TL,NaN,: Hayır,NaN,: 86903439,: 31 Temmuz 24,: Satılık Daire
2,: 1702,: 1,: 5-10,NaN,: 7,: Hayır,: Evet,: Doğalgaz (Kombi),: 12,: Evet,: Mülk Sahibi,: Yok,: 82m2,: 2+1,2.400.000 TL,: Evet,: Hayır,NaN,: 38180664,: 29 Temmuz 24,: Satılık Daire
3,,: 2,: 4,: 2,: 5,: Hayır,: Hayır,: Doğalgaz (Kombi),: 5,: Evet,: Boş,: Yok,: 180m2,: 4+2,5.150.000 TL,: Hayır,: Hayır,: Var,: 34448355,: 31 Temmuz 24,: Satılık Daire
4,,: 1,: 16-20,: 2,: Yüksek Giriş,: Hayır,: Hayır,: Doğalgaz (Kombi),: 5,: Evet,: Boş,: Yok,: 75m2,: 1+1,2.000.000 TL,: Hayır,: Hayır,: Var,: 34118702,: 30 Temmuz 24,: Satılık Daire


In [ ]:
df['Şehir']='Istanbul'

In [ ]:
df

,Aidat (TL),Banyo Sayısı,Bina Yaşı,Bir Kattaki Daire,Bulunduğu Kat,Eşyalı,Gör. Arama İle Gez.,Isıtma,Kat Sayısı,Krediye Uygun,Kullanım Durumu,Kısıtlama,Metre Karesi (Net),Oda Sayısı,Price,Site İçerisinde,Takaslı,Yapı Kullanım İzin D.,İlan No,İlan Tarihi,İlan Türü,Şehir
0,NaN,: 1,: 26-30,NaN,: 3,NaN,NaN,: Doğalgaz (Kombi),: 3,: Hayır,: Boş,: Yok,: 95m2,: 2+1,2.600.000 TL,NaN,: Hayır,NaN,: 29664799,: 31 Temmuz 24,: Satılık Daire,Istanbul
1,NaN,: 1,: 21-25,NaN,: Giriş Katı,NaN,NaN,: Kalorifer (Doğalgaz),: 6,: Hayır,: Mülk Sahibi,: Yok,: 75m2,: 2+1,1.700.000 TL,NaN,: Hayır,NaN,: 86903439,: 31 Temmuz 24,: Satılık Daire,Istanbul
2,: 1702,: 1,: 5-10,NaN,: 7,: Hayır,: Evet,: Doğalgaz (Kombi),: 12,: Evet,: Mülk Sahibi,: Yok,: 82m2,: 2+1,2.400.000 TL,: Evet,: Hayır,NaN,: 38180664,: 29 Temmuz 24,: Satılık Daire,Istanbul
3,,: 2,: 4,: 2,: 5,: Hayır,: Hayır,: Doğalgaz (Kombi),: 5,: Evet,: Boş,: Yok,: 180m2,: 4+2,5.150.000 TL,: Hayır,: Hayır,: Var,: 34448355,: 31 Temmuz 24,: Satılık Daire,Istanbul
4,,: 1,: 16-20,: 2,: Yüksek Giriş,: Hayır,: Hayır,: Doğalgaz (Kombi),: 5,: Evet,: Boş,: Yok,: 75m2,: 1+1,2.000.000 TL,: Hayır,: Hayır,: Var,: 34118702,: 30 Temmuz 24,: Satılık Daire,Istanbul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,: 480,: 2,: 4,: 5+,: 9,: Hayır,: Hayır,: Merkezi Sistem (Pay Ölçer),: 17,: Evet,: Boş,: Yok,: 130m2,: 3+1,4.300.000 TL,: Evet,: Hayır,: Var,: 34585993,: 08 Kasım 21,: Satılık Daire,Istanbul
419,,: 1,: 21-25,,: Yüksek Giriş,,,: Doğalgaz Sobası,: 6,: Evet,: Kiracılı,: Yok,: 50m2,: 1+1,325.000 TL,,: Hayır,,: 21709228,: 14 Aralık 20,: Satılık Daire,Istanbul
420,,: 2,: 30+,,: 4,,,: Doğalgaz (Kombi),: 4,: Hayır,: Kiracılı,: Yok,: 165m2,: 4+1,11.000.000 TL,,: Hayır,,: 67190883,: 13 Kasım 20,: Satılık Daire,Istanbul
421,,: 1,: 30+,,: 1,,,: Merkezi Sistem,: 5,: Evet,: Kiracılı,: Yok,: 80m2,: 2+1,4.400.000 TL,,: Hayır,,: 91296572,: 26 Ekim 20,: Satılık Daire,Istanbul


In [ ]:
df.to_csv("emlak_sitem_istanbul.csv")

## Izmir city houses

In [ ]:
PAGE_NUM = []
def find_page_num():
    for i in range(1,29):
        PAGE_NUM.append("https://emlaksitem.com/satilik-daire/izmir?page=" + str(i))
    return PAGE_NUM
find_page_num()

['https://emlaksitem.com/satilik-daire/izmir?page=1',
 'https://emlaksitem.com/satilik-daire/izmir?page=2',
 'https://emlaksitem.com/satilik-daire/izmir?page=3',
 'https://emlaksitem.com/satilik-daire/izmir?page=4',
 'https://emlaksitem.com/satilik-daire/izmir?page=5',
 'https://emlaksitem.com/satilik-daire/izmir?page=6',
 'https://emlaksitem.com/satilik-daire/izmir?page=7',
 'https://emlaksitem.com/satilik-daire/izmir?page=8',
 'https://emlaksitem.com/satilik-daire/izmir?page=9',
 'https://emlaksitem.com/satilik-daire/izmir?page=10',
 'https://emlaksitem.com/satilik-daire/izmir?page=11',
 'https://emlaksitem.com/satilik-daire/izmir?page=12',
 'https://emlaksitem.com/satilik-daire/izmir?page=13',
 'https://emlaksitem.com/satilik-daire/izmir?page=14',
 'https://emlaksitem.com/satilik-daire/izmir?page=15',
 'https://emlaksitem.com/satilik-daire/izmir?page=16',
 'https://emlaksitem.com/satilik-daire/izmir?page=17',
 'https://emlaksitem.com/satilik-daire/izmir?page=18',
 'https://emlaksite

In [ ]:
ALL_PRODUCT = []
def all_product():
    for i in PAGE_NUM[::]:
        html = getAndParseURL(i)
        for sonuc in html.findAll("a",{"class":"classified-title"}):
            ALL_PRODUCT.append("https://emlaksitem.com" + sonuc.get("href"))
    return ALL_PRODUCT
all_product()

['https://emlaksitem.com/ilan/acilacilkucuk-cigli-merkezde-sifir-satilik-11-daire-568471',
 'https://emlaksitem.com/ilan/acilacilmenemen-carsida-sifir-31-150-m2-satilik-luks-daire-573238',
 'https://emlaksitem.com/ilan/bornova-ergene-mah-m-kemal-cad-uzeri-satilik-daire-kanit-emlaktan-573526',
 'https://emlaksitem.com/ilan/muglalilaremlaktan-aydin-mah-ara-kat-esinin-dairesatilik-569551',
 'https://emlaksitem.com/ilan/a-plus-gayrimenkulden-turan-mah-sifir-dubleks-21-satilik-daire-539826',
 'https://emlaksitem.com/ilan/a-plus-gayrimenkulden-bulvar-ustu-31-satilik-daire-570595',
 'https://emlaksitem.com/ilan/izmir-menemen-esatpasa-mh-satilik-21-arakat-daire-570972',
 'https://emlaksitem.com/ilan/maliyeciler-mahde-asansorlu-otoparkli-31-satilik-daire-572810',
 'https://emlaksitem.com/ilan/cesme-reisdere-21-satilik-daire-570977',
 'https://emlaksitem.com/ilan/oyak-yamacevlerde-satilik-deniz-manzarali-31-daire-570611',
 'https://emlaksitem.com/ilan/bergama-fatih-mah-sifir-genis-kapali-mutfak-

In [ ]:


def parse_data():
    rows = []  # We'll create a row for each product
    columns = set()  # Using a set to store column names

    for url in ALL_PRODUCT:
        result = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3.1 Safari/605.1.15"})
        soup = BeautifulSoup(result.text, 'html.parser')

        target_div = soup.find("div", {"id": "ilan-bilgileri"})
        row_data = {}  # We'll store row data for each URL

        if target_div:
            ul = target_div.find('ul')
            if ul:
                li_items = ul.find_all('li')

                for li in li_items:
                    strong_items = li.find_all("strong")
                    span_items = li.find('span')

                    if len(strong_items) == 2:
                        column_name = strong_items[0].text.strip()
                        value = strong_items[1].text.strip()
                    else:
                        column_name = strong_items[0].text.strip()
                        value = span_items.text.strip() if span_items else ''

                    row_data[column_name] = value
                    columns.add(column_name)  # Add column names to the set

        prices = soup.select('span.btn.btn-success')
        price_values = [price.text.strip() for price in prices]

        if price_values:
            row_data['Price'] = ', '.join(price_values)
            columns.add('Price')  # Add 'Price' column to the set

        # Add empty data for missing columns
        for column in columns:
            if column not in row_data:
                row_data[column] = ''

        rows.append(row_data)

    # Create DataFrame
    df2 = pd.DataFrame(rows, columns=sorted(columns))
    return df2

df2 = parse_data()


In [ ]:
df2['Şehir']='Izmir'

In [ ]:
df2.to_csv("emlak_sitem_izmir.csv")

In [ ]:
df2

,Aidat (TL),Banyo Sayısı,Bina Yaşı,Bir Kattaki Daire,Bulunduğu Kat,Eşyalı,Gör. Arama İle Gez.,Isıtma,Kat Sayısı,Krediye Uygun,Kullanım Durumu,Kısıtlama,Metre Karesi (Net),Oda Sayısı,Price,Site İçerisinde,Takaslı,Yapı Kullanım İzin D.,İlan No,İlan Tarihi,İlan Türü,Şehir
0,NaN,: 1,: 0,: 2,: 1,: Hayır,: Hayır,: Doğalgaz (Kombi),: 3,: Evet,: Boş,: Yok,: 68m2,: 1+1,2.000.000 TL,: Hayır,: Hayır,: Var,: 35150895,: 23 Nisan 24,: Satılık Daire,Izmir
1,NaN,: 2,: 0,: 5+,: 3,: Hayır,: Hayır,: Doğalgaz (Kombi),: 11,: Evet,: Boş,: Yok,: 130m2,: 3+1,4.200.000 TL,: Evet,: Hayır,: Var,: 35731904,: 24 Temmuz 24,: Satılık Daire,Izmir
2,: 100,: 1,: 30+,: 3,: 2,: Hayır,: Hayır,: Yok,: 8,: Evet,: Boş,: Yok,: 88m2,: 2+1,3.600.000 TL,: Hayır,: Hayır,: Var,: 35718397,: 31 Temmuz 24,: Satılık Daire,Izmir
3,: VAR,: 1,: 30+,: 4,: 2,: Hayır,: Evet,: Doğalgaz Sobası,: 3,: Evet,: Mülk Sahibi,: Yok,: 70m2,: 1+1,1.500.000 TL,: Hayır,: Hayır,: Var,: 35178974,: 10 Mayıs 24,: Satılık Daire,Izmir
4,,: 2,: 0,,: 3,: Hayır,: Hayır,: Doğalgaz (Kombi),: 3,: Evet,: Boş,: Yok,: 130m2,: 2+1,2.500.000 TL,: Hayır,: Hayır,: Var,: 35577081,: 24 Haziran 23,: Satılık Daire,Izmir
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,: 100,: 1,: 11-15,: 5+,: 9,: Hayır,,: Doğalgaz (Kombi),: 9,: Evet,: Mülk Sahibi,: Yok,: 85m2,: 2+1,1.750.000 TL,: Evet,: Hayır,: Var,: 35284429,: 27 Mayıs 21,: Satılık Daire,Izmir
696,: 150,: 2,: 5-10,: 5+,: 6,: Hayır,: Hayır,: Doğalgaz (Kombi),: 8,: Evet,: Mülk Sahibi,: Yok,: 130m2,: 3+1,3.150.000 TL,: Evet,: Hayır,: Var,: 35663884,: 11 Mart 21,: Satılık Daire,Izmir
697,: Belirtilmemiş,: 2,: 5-10,: 2,: 2,: Hayır,: Hayır,: Doğalgaz (Kombi),: 5,: Evet,: Boş,: Yok,: 125m2,: 3+1,3.850.000 TL,: Hayır,: Hayır,,: 35703616,: 13 Ağustos 20,: Satılık Daire,Izmir
698,,: 2,: 21-25,: 1,: 1,: Hayır,,: Kat Kaloriferi,: 2,: Evet,: Boş,: Yok,: 110m2,: 3+1,16.000.000 TL,: Hayır,: Hayır,: Var,: 35324896,: 12 Mart 20,: Satılık Daire,Izmir


In [ ]:
df1=pd.read_csv("emlak_sitem_istanbul.csv")

## Concat the data.

In [ ]:
df=[]

In [ ]:
df=pd.concat([df1,df2])

In [ ]:
df.to_csv("emlak_sitem.csv")

In [ ]:
df.shape

(1123, 23)